## Huggingface의 transformer 이용하기!

[깃헙](https://github.com/huggingface/transformers)

In [ ]:
pip install transformers

     |████████████████████████████████| 3.1 MB 7.0 MB/s 
     |████████████████████████████████| 56 kB 6.3 MB/s 
     |████████████████████████████████| 596 kB 47.7 MB/s 
     |████████████████████████████████| 895 kB 58.8 MB/s 
     |████████████████████████████████| 3.3 MB 19.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import pipeline
# 감성분석 바로 진행하기
classifier = pipeline('sentiment-analysis', framework='tf')
classifier('We are very happy to include pipeline into the transformers repository.')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9978194236755371}]

## NLP framework가 NLP 모델을 통해 문제를 푸는 과정
1) Task를 정의하고 그에 맞게 dataset을 가공  
2) 적당한 model을 선택하고 이를 만듦   
3) model에 데이터들을 태워서 학습을 시킴  
4) 이를 통해 나온 weight와 설정(config)들을 저장  
5) 저장한 model의 checkpoint는 배포

## Transformers 패키지 설계구조

task를 정의하고 dataset을 알맞게 가공하는 `Processors`, 텍스트 데이터를 전처리할 수 있는 `Tokenizer`, 다양한 model을 정의한 `Model`, optimizer와 학습 schedule(warm up 등)을 관리할 수 있는 `Optimization`, 학습 과정을 전반을 관리하는 `Trainer`, weight와 tokenizer, model을 쉽게 불러올 수 있도록 각종 설정을 저장하는 `Config` 등으로 이루어져 있습니다.


# huggingface 실습

## Transformers 패키지에서 pretrained model 불러오기!
1) **task에 적합한 모델을 직접 선택하여 import**
- `from_pretrained` 메소드 사용
- 직접 학습시킨 모델도 불러올 수 있음
- Huggingface에서 제공하는 pretrained 모델이라면 모델의 이름을 string으로, 직접 학습시킨 모델이라면 config와 모델을 저장한 경로를 string으로 넘겨주면 됨

2) **Automodel 이용**
- 모델에 관한 정보 명시할 필요가 없음


>  Model ID를 통하여 pretrained model 결정

### 모델 종류에 따른 구분
- bert-base-uncased : BERT 모델인데, 108MB 파라미터의 기본 모델이면서, 코퍼스는 영문 대소문자 구분을 없앴다(전체 소문자화)
- bert-large-cased : BERT 모델인데, 340MB 파라미터의 대형 모델이면서, 코퍼스는 영문 대소문자 구분을 유지했다.
- bert-base-multilingual-cased : BERT 모델인데, 108MB 파라미터의 기본 모델이면서, 코퍼스는 다국어 대소문자 구분 유지했다.

In [ ]:
# BERT pretrained model import
from transformers import TFBertForPreTraining
model = TFBertForPreTraining.from_pretrained('bert-base-cased') #Model ID

print(model.__class__)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForPreTraining.

All the layers of TFBertForPreTraining were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForPreTraining for predictions without further training.


<class 'transformers.models.bert.modeling_tf_bert.TFBertForPreTraining'>


In [ ]:
# import Automodel
from transformers import TFAutoModel
model = TFAutoModel.from_pretrained("bert-base-cased") # Model ID

print(model.__class__)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


<class 'transformers.models.bert.modeling_tf_bert.TFBertModel'>


모델마다 그 구조는 다르지만 대부분 해당 모델 이름을 가진 클래스(eg. TFBertModel)과 MainLayer class(eg. TFBertMainLayer)와 Attention Class, Embedding Class 로 이루어져 있음.  
Automodel은 input 과 output shape가 달라질 수 있음.  
정확한 용도로 사용하려면, 메소드 안에 구성된 뼈대 `__init__()` 확인을 통해 최적의 모델을 선택하는 것이 좋음. 



## Tokenizer

- 내가 사용할 모델이 어떤 tokenizer를 쓰는지 체크
- model을 사용할 때 명시했던 것과 **동일한 ID로 tokenizer를 생성**

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [ ]:
# input, token , attention mask 까지 확인 가능한 객체 생성
encoded = tokenizer("Long Long time ago, there was a beautiful woman named Snow White.")
print(encoded)

{'input_ids': [101, 3261, 3261, 1159, 2403, 117, 1175, 1108, 170, 2712, 1590, 1417, 8442, 2061, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
# batch 단위로 input을 받을 수 있음
batch_sentences = ["Hello I'm a single sentence",
                    "And another sentence",
                    "And the very very last one"]

encoded_batch = tokenizer(batch_sentences)
print(encoded_batch)

{'input_ids': [[101, 8667, 146, 112, 182, 170, 1423, 5650, 102], [101, 1262, 1330, 5650, 102], [101, 1262, 1103, 1304, 1304, 1314, 1141, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
# 다양한 옵션 설정 가능 (padding, truncation)
batch = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="tf")
print(batch)

{'input_ids': <tf.Tensor: shape=(3, 9), dtype=int32, numpy=
array([[ 101, 8667,  146,  112,  182,  170, 1423, 5650,  102],
       [ 101, 1262, 1330, 5650,  102,    0,    0,    0,    0],
       [ 101, 1262, 1103, 1304, 1304, 1314, 1141,  102,    0]],
      dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(3, 9), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(3, 9), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 0]], dtype=int32)>}


## Processor
- **raw data를 가공하여 model에 태울 수 있는 형태를 만들어주는 작업을 해주는 클래스**
- 직접 수집한 데이터를 가공하고 싶다면, 데이터에 알맞은 processor를 직접 정의
- Task 별 복잡한 데이터 전처리 작업을 수행하는 processor를 직접 만드실 때는, DataProcessor를 상속받아서 만듦

> Bert만 하더라도,  
1)두 개의 문장을 골라서, Next Sentemce Prediction을 위해 적절히 배치하고,  
2)15%의 마스킹 포지션을 골라내기 위한 복잡한 과정을 거친 후, 심지어 그 마킹 대상도 10%, 10%의 확률로 마스킹 대신 다른 예외 처리를 해주어야 하고,  
3)Next Sentence Prediction을 위해 Segment Embedding을 위한 tensor를 따로 마련해야 합니다.

이런 과정을 위해 Huggingface에서는 `Processor`라는 추상 클래스를 추가했습니다.


아래는 Processor 중 Sequence Classification 태스크를 위한 추상 클래스인 DataProcessor의 코드 예제입니다! 함께 보시죠.

In [ ]:
class DataProcessor:
    """sequence classification을 위해 data를 처리하는 기본 processor"""

    def get_example_from_tensor_dict(self, tensor_dict):
        """
        tensor dict에서 example을 가져오는 메소드
        """
        raise NotImplementedError() # 향후 지워즈는 것을 잊지 말것 . 추상 클래스에서 꼭 구현할 부분이 Notimplemented로 남아있다면 우리가 짠 Processor는 Tokenizer와 협력하여 정상 작동하지 않을것

    def get_train_examples(self, data_dir):
        """train data에서 InputExample 클래스를 가지고 있는 것들을 모으는 메소드"""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """dev data(validation data)에서 InputExample 클래스를 가지고 있는 것들을 모으는 메소드"""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """test data에서 InputExample 클래스를 가지고 있는 것들을 모으는 메소드"""
        raise NotImplementedError()

    def get_labels(self):
        """data set에 사용되는 라벨들을 리턴하는 메소드"""
        raise NotImplementedError()

    def tfds_map(self, example):
        """
        tfds(tensorflow-datasets)에서 불러온 데이터를 DataProcessor에 알맞게 가공해주는 메소드
        """
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    @classmethod # 인스턴스 생성 없이 직접 생성 가능하게 해줌
    def _read_tsv(cls, input_file, quotechar=None):
        """tab으로 구분된 .tsv파일을 읽어들이는 클래스 메소드"""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

## Config
- 모델 학습을 위해 json file로 명시
-  batch size, learning rate, weight_decay등 train에 필요한 요소들부터, tokenizer에 특수 토큰(special token eg.[MASK])들을 미리 설정하는 등 설정에 관한 전반적인 것들이 명시

- PretrainedModel을 `save_pretrained` 메소드를 이용하면 모델의 체크포인트와 함께 저장되도록 되어있음

- 그대로 불러와서 쓰는 건 상관없으나, 설정을 변경하고 싶거나 나만의 모델 학습을 위해서는 config file을 직접 불러와야함.

- config 또한 Model ID로 클래스를 지정하거나, `AutoConfig` 이용하는 방식도 가능



In [12]:
from transformers import BertConfig # BertConfig 클래스 지정

config = BertConfig.from_pretrained("bert-base-cased") 
print(config.__class__)
print(config)

<class 'transformers.models.bert.configuration_bert.BertConfig'>
BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



In [13]:
from transformers import AutoConfig #AutoConfig

config = AutoConfig.from_pretrained("bert-base-cased")
print(config.__class__)
print(config)

<class 'transformers.models.bert.configuration_bert.BertConfig'>
BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



`model.cofig` 이용한 방법도 있다!

In [14]:
model = TFBertForPreTraining.from_pretrained('bert-base-cased')

config = model.config
print(config.__class__)
print(config)

All model checkpoint layers were used when initializing TFBertForPreTraining.

All the layers of TFBertForPreTraining were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForPreTraining for predictions without further training.


<class 'transformers.models.bert.configuration_bert.BertConfig'>
BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



## Trainer
- 말 그대로 모델 학습을 위한 클래스
- training, fine-tuning, evaluation 모두 가능함

### Tensorflow로 활용
- tensorflow의 경우 `tf.keras.model` API를 이용하여서도 `Huggingface`를 통해 불러온 모델을 활용해 학습이나 테스트를 진행할 수 있음
  - `model.fit`, `model.predict`

- `TFTrainer`를 이용할 경우, `TrainingArguments` 를 통해 Huggingface 프레임워크에서 제공하는 기능들을 통합적으로 커스터마이징하여 모델을 학습시킬 수 있다는 장점이 있음




In [15]:
# huggingface로 불러온 모델을 tf.keras.model API를 이용해 활용
import tensorflow as tf
from transformers import TFAutoModelForPreTraining, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
model = TFAutoModelForPreTraining.from_pretrained('bert-base-cased')

sentence = "Hello, This is test for bert TFmodel."

input_ids = tf.constant(tokenizer.encode(sentence, add_special_tokens=True))[None, :]  # Batch size 1

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss)
pred = model.predict(input_ids)

print("=====Results=====")
print(pred)

All model checkpoint layers were used when initializing TFBertForPreTraining.

All the layers of TFBertForPreTraining were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForPreTraining for predictions without further training.


=====Results=====
TFBertForPreTrainingOutput(loss=None, prediction_logits=array([[[ -7.402722 ,  -7.36266  ,  -7.450014 , ...,  -6.1955214,
          -5.894807 ,  -6.367268 ],
        [ -7.828724 ,  -8.058228 ,  -7.864205 , ...,  -6.419409 ,
          -6.3024373,  -6.7624674],
        [-11.549931 , -11.551907 , -11.484695 , ...,  -8.114804 ,
          -8.314194 ,  -9.444447 ],
        ...,
        [ -3.2660682,  -3.7416453,  -2.5797966, ...,  -4.0110006,
          -2.4964385,  -3.0753903],
        [-12.23197  , -12.027053 , -11.797834 , ...,  -8.838846 ,
          -9.091652 , -10.497258 ],
        [-10.639946 , -11.07434  , -11.0361   , ...,  -8.148461 ,
          -9.585203 , -10.671508 ]]], dtype=float32), seq_relationship_logits=array([[ 1.6309208, -0.7168475]], dtype=float32), hidden_states=None, attentions=None)


- `TFTrainer` 활용할 경우 학습에 필요한 arguments을 `TFTrainingArguments`을 통해서 정의
- model 생성 시에 `training_args`의 scope 안에서 진행했다는 것에 유념


In [16]:
# TFtrainer 활용
from dataclasses import dataclass, field
from enum import Enum
from typing import Dict, Optional
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import (
    TFAutoModelForSequenceClassification,
    TFTrainer,
    TFTrainingArguments,
    AutoConfig,
    AutoTokenizer,
    glue_convert_examples_to_features,
)

# TFTrainingArguments 정의
training_args = TFTrainingArguments(
    output_dir='./results',              # output이 저장될 경로
    num_train_epochs=1,              # train 시킬 총 epochs
    per_device_train_batch_size=16,  # 각 device 당 batch size
    per_device_eval_batch_size=64,   # evaluation 시에 batch size
    warmup_steps=500,                # learning rate scheduler에 따른 warmup_step 설정
    weight_decay=0.01,                 # weight decay
    logging_dir='./logs',                 # log가 저장될 경로
    do_train=True,                        # train 수행여부
    do_eval=True,                        # eval 수행여부
    eval_steps=1000
)

# model, tokenizer 생성
model_name_or_path = 'bert-base-uncased'
with training_args.strategy.scope():    # training_args가 영향을 미치는 model의 범위를 지정
    model = TFAutoModelForSequenceClassification.from_pretrained(
            model_name_or_path,
            from_pt=bool(".bin" in model_name_or_path),
        )
tokenizer = AutoTokenizer.from_pretrained(
        model_name_or_path,
    )

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [17]:
# 데이터셋 생성
ds, info = tfds.load('glue/mrpc', with_info=True)
train_dataset = glue_convert_examples_to_features(ds['train'], tokenizer, 128, 'mrpc')
train_dataset = train_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['train'].num_examples))

# TFTrainer 생성
trainer = TFTrainer(
    model=model,                          # 학습시킬 model
    args=training_args,                  # TFTrainingArguments을 통해 설정한 arguments
    train_dataset=train_dataset,   # training dataset
)

# 학습 진행
trainer.train()

# 테스트
test_dataset = glue_convert_examples_to_features(ds['test'], tokenizer, 128, 'mrpc')
test_dataset = test_dataset.apply(tf.data.experimental.assert_cardinality(info.splits['test'].num_examples))
trainer.evaluate(test_dataset)

INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: glue/mrpc/1.0.0
INFO:absl:Load dataset info from /tmp/tmpo884jqxwtfds
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.citation from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.location from disk and from code do not match. Keeping the one from code.
INFO:absl:Generating dataset glue (/root/tensorflow_datasets/glue/mrpc/1.0.0)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

INFO:absl:Downloading https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc into /root/tensorflow_datasets/downloads/fire.goog.com_v0_b_mtl-sent-repr.apps.com_o_2FjSIMlCiqs1QSmIykr4IRPnEHjPuGwAz5i40v8K9U0Z8.tsvalt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc.tmp.3fc2c429bb134c139770cf3cffc05a81...
INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt into /root/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_test0PdekMcyqYR-w4Rx_d7OTryq0J3RlYRn4rAMajy9Mak.txt.tmp.37d97c27f96c49b480ef87263b2ef78e...
INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt into /root/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_trainfGxPZuQWGBti4Tbd1YNOwQr-OqxPejJ7gcp0Al6mlSk.txt.tmp.3d01ab03b7114d1b96468bb3ca5bc4ff...
INFO:absl:Generating split train


0 examples [00:00, ? examples/s]

...

  0%|          | 0/1725 [00:00<?, ? examples/s]

INFO:absl:Done writing /root/tensorflow_datasets/glue/mrpc/1.0.0.incompleteN1B47V/glue-test.tfrecord. Shard lengths: [1725]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.SKIP.
INFO:absl:Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/glue/mrpc/1.0.0


Dataset glue downloaded and prepared to /root/tensorflow_datasets/glue/mrpc/1.0.0. Subsequent calls will reuse this data.


/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)
/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:175: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)
/usr/local/lib/python3.7/dist-packages/transformers/trainer_tf.py:114: FutureWarning: The class `TFTrainer` is de

{'eval_loss': 0.6749641277171947}

- 이후 데이터셋을 생성하여, model, training_args과 함께 TFTrainer에 전달하는 것으로 학습을 위한 준비가 마무리
- trainer.train()을 호출하면 실제 학습이 진행